In [2]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dmitriy Drozhzhin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import cross_val_score

In [4]:
data_train =  pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
data_train.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [6]:
data_train =  pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

data_train_total_entries = data_train[data_train.columns[0]].count()
for col in data_train.columns:
    if data_train[col].isna().sum() / data_train_total_entries > 0.1:
        data_train = data_train.drop(col, axis=1)
    else:
        data_train[col] = data_train[col].fillna(data_train[col].mode()[0])

data_train.info()
for i in zip(data_train.isna(), data_train.isna().sum()):
    if i[1] != 0:
        print(i[0], i[1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [7]:
for i in zip(data_train.isna(), data_train.isna().sum()):
    if i[1] != 0:
        print(i[0], i[1])

In [8]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [9]:
data_train.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [10]:
col_names = list(data_train)[:-1]
col_names

['Id',
 'MSSubClass',
 'MSZoning',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolA

In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

for col in data_train.columns:
    data_train[col] = encoder.fit_transform(data_train[col])

In [12]:
X = data_train.drop(['SalePrice'],axis=1)
X

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,5,3,327,1,3,3,0,4,0,...,49,0,0,0,0,0,1,2,8,4
1,1,0,3,498,1,3,3,0,2,0,...,0,0,0,0,0,0,4,1,8,4
2,2,5,3,702,1,0,3,0,4,0,...,30,0,0,0,0,0,8,2,8,4
3,3,6,3,489,1,0,3,0,0,0,...,24,108,0,0,0,0,1,0,8,0
4,4,5,3,925,1,0,3,0,2,0,...,70,0,0,0,0,0,11,2,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,5,3,267,1,3,3,0,4,0,...,28,0,0,0,0,0,7,1,8,4
1456,1456,0,3,866,1,3,3,0,4,0,...,0,0,0,0,0,0,1,4,8,4
1457,1457,6,3,415,1,3,3,0,4,0,...,48,0,0,0,0,17,4,4,8,4
1458,1458,0,3,505,1,3,3,0,4,0,...,0,38,0,0,0,0,3,4,8,4


In [13]:
y = data_train['SalePrice']
y

0       412
1       339
2       442
3       194
4       494
       ... 
1455    314
1456    415
1457    527
1458    199
1459    221
Name: SalePrice, Length: 1460, dtype: int64

In [14]:
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, random_state= 10)

In [15]:
# Проверьте количество образцов в тренировочных данных
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

# Проверьте количество образцов в тестовых данных
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1168, 73)
y_train shape: (1168,)
X_test shape: (292, 73)
y_test shape: (292,)


In [16]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

mse = mean_squared_error(y_pred,y_test)
print("Mean Squared Error:", mse)

cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

# Выводим среднюю оценку R^2 по всем фолдам
print("Cross-validated R^2:", np.mean(cv_scores))

R^2 Score: 0.9016611365863101
Mean Squared Error: 3006.5787357772438
Cross-validated R^2: 0.9004470768330807


In [17]:
y_test

854     298
381     362
816     180
577     278
35      574
       ... 
970     171
598     432
1058    599
1018    265
387     137
Name: SalePrice, Length: 292, dtype: int64

In [18]:
y_pred[3]

274.1283955135807